In [1]:
import pandas as pd
import numpy as np
df = pd.read_csv('https://raw.githubusercontent.com/ormarketing/b2b/master/data.csv')

In [2]:
df.Stage.unique()

array(['Closed Lost', 'Closed Won - Paid', 'Closed Won - Not Paid',
       'Closed - Lost', 'Closed Lost - Not Renewing'], dtype=object)

In [3]:
# Label rows: ['Closed Lost','Closed - Lost','Closed Lost - Not Renewing'] = 0, ['Closed Won - Not Paid','Closed Won - Paid'] = 1

conditions = [
  (df['Stage'] == 'Closed Lost'),
  (df['Stage'] == 'Closed - Lost'),
  (df['Stage'] == 'Closed Lost - Not Renewing'),
  (df['Stage'] == 'Closed Won - Not Paid'),
  (df['Stage'] == 'Closed Won - Paid')
]
values = [0,0,0,1,1]
df['y'] = np.select(conditions, values)

In [4]:
# Drop emty rows
df = df[df[' Net Amount '] != '       ']
df = df[df[' Amount '] != '       ']

In [5]:
# Change data type to 'int'
df[' Net Amount '] = df[' Net Amount '].astype(str).astype(int)
df[' Amount '] = df[' Amount '].astype(str).astype(int)


In [6]:
df['Discount'] = df[' Net Amount ']/df[' Amount ']

In [7]:
len(df)

1455

In [8]:
df

,ID,Close Date,Created Date,New Logo,Type,Stage,Billing Plan,ACV,Discount Amount,Amount,Net Amount,y,Discount
0,1,5/26/17,5/4/17,1,Direct - Cloud Product,Closed Lost,3 Years,431370,,225000,225000,0,1.000000
1,2,12/29/16,11/29/16,1,Partner - OnPremise Product,Closed Won - Paid,3 Years,22050,80850,147000,66150,1,0.450000
2,3,12/29/16,12/9/16,1,Partner - Cloud Product,Closed Won - Paid,3 Years,32750,43200,96000,52800,1,0.550000
3,4,9/9/16,4/20/16,1,Direct - Cloud Product,Closed Won - Paid,4 Years,36694,44800,128000,83200,1,0.650000
4,5,9/22/16,9/1/16,1,Partner - OnPremise Product,Closed Won - Paid,3 Years,18700,9900,66000,56100,1,0.850000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1500,1501,2/4/16,9/17/15,0,Direct - OnPremise Product,Closed Lost,NaN,17900,NaN,17900,17900,0,1.000000
1501,1502,10/4/16,2/5/16,0,Partner - OnPremise Product,Closed Lost,NaN,60000,NaN,60000,60000,0,1.000000
1502,1503,11/4/16,9/21/16,0,Direct - OnPremise Product,Closed Lost,Annual,55000,20000,75000,55000,0,0.733333
1503,1504,9/6/17,5/24/17,1,Direct - OnPremise Product,Closed Lost,NaN,32000,NaN,32000,32000,0,1.000000


In [9]:
dates = df['Close Date'].str.split("/", n = 2, expand = True)
df["Month"] = dates[0]
df["Date"] = dates[1]
df["Year"] = dates[2]
df['Month'] = df['Month'].astype(str).astype(int)
df['Date'] = df['Date'].astype(str).astype(int)
df['Year'] = df['Year'].astype(str).astype(int)

In [10]:
end_of_the_quarter = []
Month = list(df['Month'])
Date = list(df['Date'])

In [11]:
for i in range(len(Month)):
  if Month[i] == 3 and Date[i] >= 15:
    end_of_the_quarter.append(1)
  elif Month[i] == 6 and Date[i] >= 15:
    end_of_the_quarter.append(1)
  elif Month[i] == 9 and Date[i] >= 15:
    end_of_the_quarter.append(1)
  elif Month[i] == 12 and Date[i] >= 15:
    end_of_the_quarter.append(1)
  else:
    end_of_the_quarter.append(0)


In [12]:
df['end_of_the_quarter'] = end_of_the_quarter

In [13]:
df['Billing Plan'].unique()

array(['3 Years', '4 Years', '2 Years', nan, 'Annual', '5 Years',
       '3 Months', '9 Months', 'Quarterly'], dtype=object)

In [14]:
# Split df
#train = df.sample(frac = 0.8)
#hold = df.drop(train.index)

split_value = int(len(df) * 0.80)
train = df[:split_value]
hold = df[split_value:]

In [15]:
# train dataset
train_x = pd.get_dummies(train[['New Logo','Type',' Net Amount ','end_of_the_quarter','Discount']], prefix=['Type'], dummy_na=True)
train_y = train['y']

In [16]:
# test dataset
hold_x = pd.get_dummies(hold[['New Logo','Type',' Net Amount ','end_of_the_quarter','Discount']], prefix=['Type'], dummy_na=True)
hold_y = hold['y']

In [17]:
from sklearn.linear_model import LogisticRegression
from sklearn import metrics 

logistic_regression = LogisticRegression()
logistic_regression.fit(train_x, train_y)
pred_y = logistic_regression.predict(hold_x)

accuracy = metrics.accuracy_score(hold_y, pred_y)
accuracy


0.718213058419244

In [25]:
from sklearn.ensemble import RandomForestClassifier
clf=RandomForestClassifier(n_estimators=100)
clf.fit(train_x,train_y)
y_un_optimized = clf.predict(hold_x)
print("Accuracy:",metrics.accuracy_score(hold_y, y_un_optimized))

Accuracy: 0.8213058419243986


In [36]:
un_optimized_revenue = sum(hold[" Amount "]*hold["Discount"]*y_un_optimized)
print("Un-optimized revenue:", un_optimized_revenue)

Un-optimized revenue: 4548153.0


In [37]:
hold_x_no_discount = pd.get_dummies(hold[['New Logo','Type',' Amount ','end_of_the_quarter','Discount']], prefix=['Type'], dummy_na=True)
y_no_discount = clf.predict(hold_x_no_discount)
no_discount_revenue = sum(hold[" Amount "]*hold["Discount"]*y_no_discount)
print("No-discount revenue:",no_discount_revenue)

No-discount revenue: 4296059.0


In [22]:
sum(hold[" Amount "]*hold["Discount"]*hold['y'])

4027782.0

In [23]:
hold['Type'].unique()

array(['Direct - OnPremise Product', 'Partner - OnPremise Product',
       'Direct - Cloud Product', 'Partner - Cloud Product'], dtype=object)

In [24]:
train_x.dtypes

New Logo                              int64
 Net Amount                           int64
end_of_the_quarter                    int64
Discount                            float64
Type_Direct - Cloud Product           uint8
Type_Direct - OnPremise Product       uint8
Type_Partner - Cloud Product          uint8
Type_Partner - OnPremise Product      uint8
Type_nan                              uint8
dtype: object